In [1]:
####importing necessary packages#######
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import wandb

In [2]:
# Initializing a new Weights & Biases run with the specified project name
project_name='Without attention'
wandb.login(key="5bfaaa474f16b4400560a3efa1e961104ed54810")
wandb.init(project=project_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aravind_iitm. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\PhysioSens\.netrc


In [2]:
train_data=pd.read_csv(r'data\mal_train.csv')  # Load training data from CSV file
val_data=pd.read_csv(r'data/mal_valid.csv')# Load validation data from CSV file
test_data=pd.read_csv(r'data/mal_test.csv')  # Load test data from CSV file

In [3]:
eng_train = train_data['English']  # Extract English training data
mal_train = train_data['Malayalam']  # Extract Malayalam training data
eng_val = val_data['English']  # Extract English validation data
mal_val = val_data['Malayalam']  # Extract Malayalam validation data
eng_test = test_data['English']  # Extract English test data
mal_test = test_data['Malayalam']  # Extract Malayalam test data
english_all = pd.concat([eng_train, eng_val, eng_test], ignore_index=True)  # Concatenate all English data
malayalam_all = pd.concat([mal_train, mal_val, mal_test], ignore_index=True)  # Concatenate all Malayalam data


In [4]:
def unique_char(lan):
    unique_cha = ''  # Initialize an empty string to store unique characters
    for word in lan:  # Iterate through each word in the list
        for char in word:  # Iterate through each character in the word
            if char not in unique_cha:  # Check if the character is not already in the unique_cha string
                unique_cha += char  # Append the unique character to the string
    return unique_cha  # Return the string containing all unique characters


In [5]:
class LANG:
    def __init__(self, lang):
        self.lang = lang  # Initialize the language data
        self.word2index = {}  # Dictionary to map characters to indices
        self.index2word = {0: 'SOS', 1: 'EOS'}  # Dictionary to map indices to characters
        self.max_length = 0  # Variable to store the length of the longest word
        self.count = 2  # Counter for indexing, starting after SOS and EOS
        self.max_word = ''  # Variable to store the longest word

    def addchar(self):
        for word in self.lang:  # Iterate through each word in the language data
            length = len(word)  # Get the length of the current word
            if length > self.max_length:  # Check if the current word is the longest so far
                self.max_length = length  # Update the maximum word length
                self.max_word = word  # Update the longest word
            for char in word:  # Iterate through each character in the word
                if char not in self.word2index:  # Check if the character is not already indexed
                    self.word2index[char] = self.count  # Assign the current count as the index
                    self.index2word[self.count] = char  # Map the current count to the character
                    self.count += 1  # Increment the counter
        return self.word2index, self.index2word, self.max_length, self.max_word  # Return the mappings and max values


In [6]:
# Initialize language objects for English and Malayalam
eng = LANG(english_all)
# Get mappings and max values for English
eng_word2index, eng_index2word, eng_maxlength, eng_word = eng.addchar()
access_eng = eng_word2index, eng_index2word, eng_maxlength, eng_word

mal = LANG(malayalam_all)
# Get mappings and max values for Malayalam
mal_word2index, mal_index2word, mal_maxlength, mal_word = mal.addchar()
access_mal = mal_word2index, mal_index2word, mal_maxlength, mal_word

In [7]:
def Tensorpair(eng, mal, access_eng, access_mal):
    # Unpack access_eng and access_mal
    eng_word2index, eng_index2word, eng_maxlength, eng_word = access_eng
    mal_word2index, mal_index2word, mal_maxlength, mal_word = access_mal
    
    n = len(eng)
    # Initialize tensors for input and target sequences
    input_ids = torch.zeros((n, eng_maxlength + 1), dtype=torch.int32)
    target_ids = torch.zeros((n, eng_maxlength + 1), dtype=torch.int32)
    
    for i, (eng_word, mal_word) in enumerate(zip(eng, mal)):
        try:
            # Convert characters to indices using word2index mappings
            input_indx = [eng_word2index[char] for char in eng_word]
            input_indx.append(1)  # Append EOS token index
            input_indx = torch.tensor(input_indx, dtype=torch.long)
            
            target_indx = [mal_word2index[char] for char in mal_word]
            target_indx.append(1)  # Append EOS token index
            target_indx = torch.tensor(target_indx, dtype=torch.long)
            
            # Update input and target tensors
            input_ids[i, :len(input_indx)] = input_indx
            target_ids[i, :len(target_indx)] = target_indx
        except Exception as e:
            print(e)  # Print any exception that occurs
    
    # Create a TensorDataset from input and target tensors
    tensor_data = TensorDataset(input_ids, target_ids)
    
    return tensor_data


In [8]:
# Generating tensor pairs for training, validation, and testing data
train_data=Tensorpair(eng_train,mal_train,access_eng,access_mal)
val_data=Tensorpair(eng_val,mal_val,access_eng,access_mal)
test_data=Tensorpair(eng_test,mal_test,access_eng,access_mal)

In [9]:
# Setting batch size and creating data loaders for training, validation, and testing
batch_size=64
train_loader=DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True,drop_last=True)
val_loader=DataLoader(dataset=val_data,batch_size=batch_size,drop_last=True)
test_loader=DataLoader(dataset=test_data,batch_size=batch_size,drop_last=True)

In [10]:
class Encoder(nn.Module):
    def __init__(self, hidden_size=512, num_layers=2, drop_out=0.2, embedding_size=256, bidirection=False, model='LSTM'):
        super().__init__()
        # Initialize the embedding layer
        self.embedding = nn.Embedding(len(eng_index2word), embedding_size)
        # Define the parameters
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.drop_out = drop_out
        # Choose the RNN model (LSTM/GRU/RNN)
        if model == 'LSTM':
            self.model = nn.LSTM(embedding_size, hidden_size, num_layers, bidirectional=bidirection, batch_first=True)
        elif model == 'GRU':
            self.model = nn.GRU(embedding_size, hidden_size, num_layers, bidirectional=bidirection, batch_first=True)
        elif model == 'RNN':
            self.model = nn.RNN(embedding_size, hidden_size, num_layers, bidirectional=bidirection, batch_first=True)
        else:
            raise ValueError('Given model is not found')
        # Apply dropout regularization
        self.drop_out = nn.Dropout(p=drop_out)
    
    def forward(self, input):
        # Embed the input sequence
        embedded = self.drop_out(self.embedding(input))
        # Pass the embedded sequence through the RNN model
        output, hidden = self.model(embedded)
        return output, hidden


In [11]:
class Decoder(nn.Module):
    def __init__(self, hidden_size=512, num_layers=2, drop_out=0.2, embedding_size=256, bidirection=False, model='LSTM'):
        super().__init__()
        # Initialize the embedding layer
        self.embedding = nn.Embedding(len(mal_index2word), embedding_size)
        # Define the parameters
        self.hidden_size = hidden_size
        self.num_layer = num_layers
        self.drop_out = drop_out
        # Choose the RNN model (LSTM/GRU/RNN)
        if model == 'LSTM':
            self.model = nn.LSTM(embedding_size, hidden_size, num_layers, bidirectional=bidirection, batch_first=True)
        elif model == 'GRU':
            self.model = nn.GRU(embedding_size, hidden_size, num_layers, bidirectional=bidirection, batch_first=True)
        elif model == 'RNN':
            self.model = nn.RNN(embedding_size, hidden_size, num_layers, bidirectional=bidirection, batch_first=True)
        else:
            raise ValueError('Given model is not found')
        # Apply dropout regularization
        self.drop_out = nn.Dropout(p=drop_out)
        # Define the output layer
        if bidirection:
            self.out = nn.Linear(2 * hidden_size, len(mal_index2word))
        else:
            self.out = nn.Linear(hidden_size, len(mal_index2word))
            
    def forward(self, input, hidden):
        # Embed the input sequence
        embedded = self.drop_out(self.embedding(input))
        # Pass the embedded sequence through the RNN model
        output, hidden = self.model(embedded, hidden)
        # Predict the output
        pred = self.out(output)
        return pred, hidden


In [17]:
class seq2seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        # Initialize the encoder and decoder
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, inputs, targets, teacher_force_ratio):
        # Pass the input sequence through the encoder
        encoder_output, encoder_hidden = self.encoder(inputs)
        # Initialize the decoder input with zeros
        decoder_input = torch.empty(targets.shape[0], 1, dtype=torch.long, device='cuda').fill_(0)
        decoder_hidden = encoder_hidden
        # Initialize the output tensor
        output = torch.zeros((targets.shape[0], targets.shape[1], len(mal_index2word)), device='cuda')
        
        for i in range(output.shape[1]):
            # Pass the decoder input and hidden state through the decoder
            pred, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            pred = torch.squeeze(pred)
            # Store the predicted output
            output[:, i, :] = pred
            # Update the decoder input for the next time step
            best_guess = torch.argmax(pred, axis=-1).view(-1, 1)
            decoder_input = best_guess if np.random.rand() > teacher_force_ratio else targets[:, i].view(-1, 1)
            # Keep the decoder hidden state unchanged
            decoder_hidden = decoder_hidden
        return output


In [18]:
def word_finder_eng(eng_):
    # Initialize an empty list to store English words
    full = []
    for eng in eng_:
        # Find the index of the end-of-sequence token (EOS)
        eng_eos = np.where(eng == 1)[0][0]
        # Convert the numerical representation to an English word
        eng_word = num_word(eng[0:eng_eos], eng_index2word)
        full.append(eng_word)
    return np.array(full)

def word_finder_mal(mal_):
    # Initialize an empty list to store Malayalam words
    full = []
    for mal in mal_:
        # Find the index of the end-of-sequence token (EOS) if present, else use the length of the sequence
        mal_eos = np.where(mal == 1)[0][0] if 1 in mal else len(mal)
        # Convert the numerical representation to a Malayalam word
        mal_word = num_word(mal[0:mal_eos], mal_index2word)
        full.append(mal_word)
    return np.array(full)

def num_word(number, converter):
    # Convert a sequence of numerical representations to a word using the provided converter
    number = np.array(number)
    word = ''.join(converter[num] for num in number)
    return word


In [17]:
def train_model(epochs=30, hidden_size=512, num_layers=3, encoder_drop_out=0.2, decoder_drop_out=0.2, embedding_size=256,
                bidirection=False, model='LSTM', lr=1e-3, optimizer_='adam', teacher_force_ratio=0.5):
    # Set hyperparameters
    epochs = epochs
    hidden_size = hidden_size
    num_layers = num_layers
    encoder_drop_out = encoder_drop_out
    decoder_drop_out = decoder_drop_out
    embedding_size = embedding_size
    bidirection = bidirection
    model = model
    lr = lr
    optimizer_ = optimizer_
    teacher_force_ratio = teacher_force_ratio

    # Initialize encoder and decoder
    encoder = Encoder(hidden_size=hidden_size, num_layers=num_layers, drop_out=encoder_drop_out, embedding_size=embedding_size,
                      bidirection=bidirection, model=model)
    decoder = Decoder(hidden_size=hidden_size, num_layers=num_layers, drop_out=decoder_drop_out, embedding_size=embedding_size,
                      bidirection=bidirection, model=model)
    # Create seq2seq model
    model = seq2seq(encoder, decoder)
    model = model.to('cuda')

    # Define loss function
    lossfun = nn.CrossEntropyLoss()

    # Choose optimizer
    if optimizer_ == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    elif optimizer_ == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    else:
        raise ValueError('Optimizer not found')

    # Initialize lists to store training and validation metrics
    train_loss, train_acc, val_loss, val_acc = [], [], [], []

    # Main training loop
    for i in range(epochs):
        # Initialize lists to store current epoch metrics
        train_loss_curr, train_acc_curr, val_loss_curr, val_acc_curr = [], [], [], []

        # Set model to training mode
        model.train()

        # Iterate over training data
        for eng, mal in tqdm(train_loader):
            eng = eng.to('cuda')
            mal = mal.to('cuda')

            # Forward pass
            pred = model(eng, mal, teacher_force_ratio=teacher_force_ratio)
            pred_loss = pred.reshape(-1, pred.shape[2])
            mal_loss = mal.reshape(-1,).long()
            loss = lossfun(pred_loss, mal_loss)

            # Compute and store training loss
            train_loss_curr.append(loss.cpu().detach().numpy())

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Compute training accuracy
            mal_ground = word_finder_mal(mal.cpu().detach().numpy())
            pred_acc = np.argmax(pred.cpu().detach().numpy(), axis=-1).astype(np.int64)
            mal_predicted = word_finder_mal(pred_acc)
            acc = np.sum(mal_ground == mal_predicted) / len(mal_ground)
            train_acc_curr.append(acc)

        # Set model to evaluation mode
        model.eval()

        # Iterate over validation data
        for eng, mal in val_loader:
            eng = eng.to('cuda')
            mal = mal.to('cuda')

            # Forward pass (no teacher forcing)
            with torch.no_grad():
                pred = model(eng, mal, teacher_force_ratio=0)

            pred_loss = pred.reshape(-1, pred.shape[2])
            mal_loss = mal.reshape(-1,).long()

            # Compute and store validation loss
            loss = lossfun(pred_loss, mal_loss)
            val_loss_curr.append(loss.cpu().detach().numpy())

            # Compute validation accuracy
            mal_ground = word_finder_mal(mal.cpu().detach().numpy())
            pred_acc = np.argmax(pred.cpu().detach().numpy(), axis=-1)
            mal_predicted = word_finder_mal(pred_acc)
            acc = np.sum(mal_ground == mal_predicted) / len(mal_ground)
            val_acc_curr.append(acc)

        # Compute average metrics for the epoch
        train_loss.append(np.average(train_loss_curr))
        val_loss.append(np.average(val_loss_curr))
        train_acc.append(np.average(train_acc_curr))
        val_acc.append(np.average(val_acc_curr))

        # Log metrics using wandb
        wandb.log({"Train_Accuracy": np.round(train_acc[i] * 100, 2), "Train_Loss": train_loss[i],
                   "Val_Accuracy": np.round(val_acc[i] * 100, 2), "Val_Loss": val_loss[i], "Epoch": i + 1})


In [16]:
# Configuration for hyperparameter tuning using Bayesian optimization method
sweep_config = {
    'method': 'bayes',  # Bayesian optimization method
    'metric': {
        'name': 'Val_Accuracy',  # Metric to optimize
        'goal': 'maximize'  # Goal: maximize the validation accuracy
    },
}

# Define the parameters to tune
parameters_dict = {
    'epochs': {
        'values': [10, 15]  # Number of epochs
    },
    'hidden_size': {
        'values': [32, 64, 256, 512]  # Hidden size of the model
    },
    'num_layers': {
        'values': [1, 2, 3]  # Number of layers in the encoder and decoder
    },
    'encoder_drop_out': {
        'values': [0.2, 0.3, 0.5]  # Dropout probability in the encoder
    },
    'decoder_drop_out': {
        'values': [0.2, 0.3, 0.5]  # Dropout probability in the decoder
    },
    'embedding_size': {
        'values': [32, 64, 256, 512]  # Size of the embedding layer
    },
    'bidirectional': {
        'values': [True, False]  # Whether to use bidirectional RNNs or not
    },
    'model': {
        'values': ['LSTM', 'GRU', 'RNN']  # Type of RNN model
    },
    'lr': {
        'values': [1e-3, 1e-4, 1e-5]  # Learning rate
    },
    'teacher_force_ratio': {
        'values': [0.2, 0.3, 0.4, 0.5]  # Teacher forcing ratio during training
    },
    'optimizer': {
        'values': ['adam', 'sgd']  # Optimizer to use
    }
}

sweep_config['parameters'] = parameters_dict  # Add parameters to the sweep configuration


In [19]:
def train():
    # Initialize Weights and Biases
    wandb.init()
    # Retrieve hyperparameters from the configuration
    config = wandb.config
    # Set a name for the run based on hyperparameters
    wandb.run.name = "hidden_" + str(config.hidden_size) + "_layer_" + str(config.num_layers) + "_embedd_" + str(config.embedding_size) + "_bidirect_" + str(config.bidirectional) + "_model_" + str(config.model)
    
    # Train the model with the specified hyperparameters
    parameters = train_model(epochs=config.epochs, hidden_size=config.hidden_size, num_layers=config.num_layers,
                             encoder_drop_out=config.encoder_drop_out, decoder_drop_out=config.decoder_drop_out,
                             embedding_size=config.embedding_size, bidirection=config.bidirectional, model=config.model,
                             lr=config.lr, optimizer_=config.optimizer, teacher_force_ratio=config.teacher_force_ratio)
    
    # Finish the Weights and Biases run
    wandb.finish()


In [20]:
wandb.init()
sweep_id=wandb.sweep(sweep_config,project=project_name)
wandb.agent(sweep_id,train)
wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: qldbwt3i
Sweep URL: https://wandb.ai/aravind_iitm/Without%20attention/sweeps/qldbwt3i


wandb: Agent Starting Run: eqh1kg1k with config:
wandb: 	bidirectional: True
wandb: 	decoder_drop_out: 0.3
wandb: 	embedding_size: 256
wandb: 	encoder_drop_out: 0.5
wandb: 	epochs: 10
wandb: 	hidden_size: 32
wandb: 	lr: 1e-05
wandb: 	model: LSTM
wandb: 	num_layers: 3
wandb: 	optimizer: sgd
wandb: 	teacher_force_ratio: 0.5
Exception in thread IntMsgThr:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 980, in _bootstrap_inner
Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 980, in _bootstrap_inner
Exception in thread NetStatThr:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 917, in run
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 917, in run
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.

  0%|          | 0/800 [00:00<?, ?it/s]

  0%|          | 0/800 [00:00<?, ?it/s]

...

  0%|          | 0/800 [00:00<?, ?it/s]

Epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
Train_Accuracy,▁▄▅▅▆▆▇▇▇▇█████
Train_Loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁
Val_Accuracy,▁▆▆▇▇█▇████████
Val_Loss,▇▃▂▃▁▃▂▄▄▆▆▇█▇█
Epoch,15
Train_Accuracy,88.37
Train_Loss,0.02222
Val_Accuracy,51.39
Val_Loss,0.48265


wandb: Ctrl + C detected. Stopping sweep.


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001668AAFA640>> (for post_run_cell):


ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

In [44]:
correct_count=0
for eng,mal in train_data:
    eng_word=word_finder_eng(eng)
    mal_word=word_finder_mal(mal)
    print(eng_word)
    print(mal_word)
    eng=eng.unsqueeze(0).to('cuda')
    mal=mal.unsqueeze(0).to('cuda')
    
    pred=model(eng,mal,teacher_force_ratio=0)
    pred=torch.squeeze(pred).cpu().detach().numpy()
    pred=np.argmax(pred,axis=-1)
    predicted=word_finder_mal(pred)
    if predicted==mal_word:
        correct_count+=1
    count+=1
    print(predicted)
    print("")
    
    

vrithiyaakkiyathinu
വൃത്തിയാക്കിയത്തിനു
വൃത്തിയാക്കിയത്തിനു

viyojichaanu
വിയോജിച്ചാണ്
വിയോജിച്ചാണ്

kuthamkutty
കുത്താംകുട്ടി
കുത്തംകുട്ടി

jabbariyenna
ജബ്ബാരിയെന്ന
ജബ്ബറിയെന്ന

senegalese
സെനെഗലീസ്
സെനെഗീലെസ്

niswanodum
നിസ്വനോടും
നിസ്വനോടും

allippidichukayari
അള്ളിപ്പിടിച്ചുകയറി
അള്ളിപ്പിടിച്ചുകയറി

kavithaamaathrukayeyo
കവിതാമാതൃകയെയോ
കവിതാമാത്രേയുകയോ

bavakutty
ബാവകുട്ടി
ബാവകുട്ടി

aalambaheenarumaaya
ആലംബഹീനരുമായ
ആലംബഹീനരുമായ

fraansileykku
ഫ്രാൻസിലേയ്ക്ക്
ഫ്രാൻസിലേയ്ക്ക്

sambhaavyamaan
സംഭാവ്യമാണ്
സംഭാവ്യമാണ്

mahasweta
മഹാശ്വേത
മഹാസ്വേത

soritta
സോറിറ്റ
സോറിറ്റ

thanichaakkiyittu
തനിച്ചാക്കിയിട്ടു
തനിച്ചാക്കിയിട്ടു

neraanonnu
നേരാണൊന്നു
നേരാണൊന്ന്

keezhadangunnavarundu
കീഴടങ്ങുന്നവരുണ്ട്
കീഴടങ്ങുന്നവരുണ്ട്

kaambasilundaakum
കാമ്പസിലുണ്ടാകും
കാമ്പസിലുണ്ടാകും

mandasmithatthilum
മന്ദസ്മിതത്തിലും
മന്ദസ്മിതത്തിലും

samarppichukondum
സമർപ്പിച്ചുകൊണ്ടും
സമർപ്പിച്ചുകൊണ്ടും

ekkarilekkulla
ഏക്കറിലേക്കുള്ള
എക്കറിലേക്കുള്ള

samarchu
സമർച്ചു
സമർച്ചു

maargathilum
മാർഗത്തിലും
മാർഗത്

ജൂഡിയായിലെ

puthanpandakashaalayil
പുത്തൻപണ്ടകശാലയിൽ
പുത്തൻപന്ടകശാലയിൽ

eravikuttanpilla
ഇരവികുട്ടൻപിള്ള
ഇരവികുട്ട്പനില്ല

aakhyaanakaaranodu
ആഖ്യാനകാരനോടു
ആഖ്യാനകാരനോട്

athaazhakkottukaarum
അത്താഴക്കൊട്ടുകാരും
അത്താഴക്കോട്ടുകാരും

vanithayozhichu
വനിതയൊഴിച്ച്
വനിതയൊഴിച്ച്

janthushas
ജന്തുശാസ്
ജന്തുശാസ്

peythirangunnilla
പെയ്തിറങ്ങുന്നില്ല
പെയ്തിറങ്ങുന്നില്ല

anumathiyekkurichum
അനുമതിയെക്കുറിച്ചും
അനുമതിയെക്കുറിച്ചും

sentarililla
സെന്ററിലില്ല
സെന്ററിലില്ല

prabalamaakkiyathu
പ്രബലമാക്കിയത്
പ്രബലമാക്കിയത്

awant
അവാന്ത്
അവാന്ത്

lijeshbabu
ലിജേഷ്ബാബു
ലിജേഷ്ബാബു

sangeethaaru
സംഗീതാര്
സംഗീതാര്

parisheelakanaakkiyathum
പരിശീലകനാക്കിയതും
പരിശീലകനാക്കിയതും

thanuppaayirunnathre
തണുപ്പായിരുന്നത്രേ
തണുപ്പായിരുന്നത്രെ

bandhamillaathavarude
ബന്ധമില്ലാത്തവരുടെ
ബന്ധമില്ലാത്തവരുടെ

kazhuthakaludeyum
കഴുതകളുടേയും
കഴുതകളുടെയും

thakaraarililla
തകരാറിലില്ല
തകരാറിലില്ല

kaulumaayulla
കൌളുമായുള്ള
കൌലുമായുള്ള

khadaani
ഖദാനി
ഖദാനി

kaikollunnathennaanu
കൈകൊള്ളുന്നതെന്നാണ്
കൈകൊള്ളുന്നതെന്നാണ്

fan

കണക്കുനോക്കാറില്ല

maattiyezhuthiyath
മാറ്റിയെഴുതിയത്
മാറ്റിയെഴുതിയത്

naalaamathetho
നാലാമത്തേതോ
നാലാമത്തേതോ

shikharathilum
ശിഖരത്തിലും
ശിഖരത്തിലും

paarthiyar
പാർത്തിയർ
പാർത്തിയർ

varanaadhikaarikalaan
വരണാധികാരികളാണ്
വരണാധികാരികളാണ്

uruttiyenn
ഉരുട്ടിയെന്ന്
ഉരുട്ടിയെന്ന്

muttiyalla
മുട്ടിയല്ല
മുട്ടിയല്ല

saamoohyavimarshanavumaayi
സാമൂഹ്യവിമർശനവുമായി
സാമൂഹ്യവിമർഷനവുമായി

pradhanyamullathumaaya
പ്രധാന്യമുള്ളതുമായ
പ്രധന്യമുള്ളതുമായ

rahasyamozhiyeduppum
രഹസ്യമൊഴിയെടുപ്പും
രഹസ്യമൊഴിയെപ്പും

annanidraaviheenaraayi
അന്നനിദ്രാവിഹീനരായി
അന്നനിദ്രാവിഹീനരായി

munnariyippavum
മുന്നറിയിപ്പാവും
മുന്നറിയിപ്പാവും

bhaaramundaakkunnu
ഭാരമുണ്ടാക്കുന്നു
ഭാരമുണ്ടാക്കുന്നു

muttukalulla
മുട്ടുകളുള്ള
മുട്ടുകളുള്ള

janathaaparttiyile
ജനതാപാർട്ടിയിലെ
ജനതാപാർട്ടിയിലെ

paatamathaanu
പാഠമതാണ്
പാഠമതാണ്

nehayodu
നേഹയോടു
നേഹയോട്

maadyamangaludam
മാധ്യമങ്ങളുടം
മാധ്യമങ്ങളുടം

anthassamsthaanapaatha
അന്തസ്സംസ്ഥാനപാത
അന്തസ്സംസ്ഥാനപത്താ

poovanaayi
പൂവനായി
പൂവനായി

endiekkundennu
എൻഡിഎക്കുണ്ടെന്ന്
എൻഡിക്കേണ്ടു

കുളിർന്നു

raanayude
റാണയുടെ
റാണയുടെ

anugrahamchorinju
അനുഗ്രഹംചൊരിഞ്ഞു
അനുഗ്രഹംചൊരിഞ്ഞു

zoolie
ജൂലി
ഓൂലി

pattelaanennath
പട്ടേലാണെന്നത്
പറ്റെലാണെന്നത്

randaalkkulla
രണ്ടാൾക്കുള്ള
രണ്ടാൾക്കുള്ള

aaltharayennum
ആൽത്തറയെന്നും
ആൽത്തറയെന്നും

varuthathum
വരുത്തതും
വരുത്തതും

yasutoshi
യസുതോഷി
യസുതോഷി

nanuthupeyyunna
നനുത്തുപെയ്യുന്ന
നനുത്തുപെയ്യുന്ന

dhyanavigraham
ധ്യാനവിഗ്രഹം
ധ്യാനവിഗ്രഹം

mccavum
മക്കാവും
മക്കാവും

chilamgokal
ചിലംഗോകൾ
ചിലംഗോകൾ

adukkivekkuka
അടുക്കിവെക്കുക
അടുക്കിവെക്കുക

vishaadamshangalum
വിശാദംശങ്ങളും
വിഷാദംശങ്ങളും

meemukale
മീമുകളെ
മീമുകളെ

akramangalennaanu
അക്രമങ്ങളെന്നാണ്
അക്രമങ്ങളെന്നാണ്

aalukalkkinnishtam
ആളുകൾക്കിന്നിഷ്ടം
ആളുകൾക്കിന്നിഷ്ടം

mikkunnuvennaanu
മിക്കുന്നുവെന്നാണ്
മിക്കുന്നുവെന്നാണ്

appukalkkaanu
അപ്പുകൾക്കാണ്
അപ്പുകൾക്കാണ്

vaykkumoyenna
വയ്ക്കുമോയെന്ന
വയ്ക്കുമോയെന്ന

kiyennathine
കിയെന്നതിനെ
കിയെന്നതിനെ

thalayillaayirunnu
തലയില്ലായിരുന്നു
തലയില്ലായിരുന്നു

glassukalude
ഗ്ലാസ്സുകളുടെ
ഗ്ലാസ്ുകളുടെ

chenthundu
ചെന്തുണ്ട്
ചെന്തുണ്ട്

embaap

മതജീവിതത്തിലേക്ക്

vannucherumenkilum
വന്നുചേരുമെങ്കിലും
വന്നുചേരുമെങ്കിലും

othukkikkalanju
ഒതുക്കിക്കളഞ്ഞു
ഒതുക്കിക്കളഞ്ഞ്

swabhaavikalaaya
സ്വഭാവികളായ
സ്വഭാവികളായ

printumahaadevu
പ്രിന്റുമഹാദേവ്
പ്രിന്റുമാാദേവ്

ravonichineyum
റാവോണിച്ചിനെയും
റാവോണിച്ചിനെയും

dhakyasahajamaayi
ധക്യസഹജമായി
ധക്യസഹജമായി

basukalethunna
ബസുകളെത്തുന്ന
ബസുകളെത്തുന്ന

vibhedanamellaam
വിഭേദനമെല്ലാം
വിഭേദനമെല്ലാം

sooryakaanthiyaanu
സൂര്യകാന്തിയാണ്
സൂര്യകാന്തിയാണ്

divyathvam
ദിവ്യത്ത്വം
ദിവ്യത്ത്വം

maanushikaparigananavachu
മാനുഷികപരിഗണനവച്ചു
മാനുഷികപരിഗാനവച്ച്

mrgeeyavaasanakale
മൃഗീയവാസനകളെ
മൃഗീയവാസനകളെ

bheekaranmaaraakkunnu
ഭീകരന്മാരാക്കുന്നു
ഭീകരന്മാരാക്കുന്നു

thanginilkkuvaan
തങ്ങിനിൽക്കുവാൻ
തങ്ങിനിൽക്കുവാൻ

vimoolam
വിമൂലം
വിമൂലം

yeshuvintethenna
യേശുവിന്റേതെന്ന
യേശുവിന്റേതെന്ന

valavumonnum
വളവുമൊന്നും
വളവുമൊന്നും

phonukalilokke
ഫോണുകളിലൊക്കെ
ഫോണുകളിലൊക്കെ

shantaladevi
ശന്തലാദേവി
ശാന്തലാവേ

checkpoyinu
ചെക്ക്പോയിന്
ചെക്ക്പോയിന്

vaadaamalaraayi
വാടാമലരായി
വാടാമലരായി

daveyaanu
ദവേയാണ്
ദവേയാ

...

അഴിഞ്ഞാടിയെന്ന

jillakkaavasyamaaya
ജില്ലക്കാവശ്യമായ
ജില്ലക്കാവ്യസമായ

pattullath
പട്ടുള്ളത്
പറ്റുള്ളത്

rakshakarthaakkalum
രക്ഷകർത്താക്കളും
രക്ഷകർത്താക്കളും

aanukoolyangalulla
ആനുകൂല്യങ്ങളുള്ള
ആനുകൂല്യങ്ങളുള്ള

mukhamandapangalile
മുഖമണ്ഡപങ്ങളിലെ
മുഖമന്ദപങ്ങളിലെ

bhaaviparanja
ഭാവിപറഞ്ഞ
ഭാവിപറഞ്ഞ

uthsavachadangaanu
ഉത്സവചടങ്ങാണ്
ഉത്സവചച്ങടങ്ങാണ്

kaaryejilum
കാര്യേജിലും
കാര്യേജിലും

vaardhakyangal
വാർദ്ധക്യങ്ങൾ
വാർദ്ധക്യങ്ങൾ

rilaayumaanu
റിലായുമാണ്
റിലായുമാണ്

bethoinecurichum
ബെഥോയിനെക്കുറിച്ചും
ബെതോയിനെക്കുറിച്ചും

virattalumonnum
വിരട്ടലുമൊന്നും
വിരട്ടലുമൊന്നും

saamoohikaikyam
സാമൂഹികൈക്യം
സാമൂഹികൈക്യം

vavidyaalayam
വവിദ്യാലയം
വവിദ്യാലയം

amarakkaaraakuka
അമരക്കാരാകുക
അമരക്കാരാകുക

roopakalppanayudeyum
രൂപകൽപ്പനയുടേയും
രൂപകല്പന്പയുടെയും

panikkokke
പണിക്കൊക്കെ
പണിക്കൊക്കെ

kambanikalaanennathu
കമ്പനികളാണെന്നത്
കമ്പനികളാണെന്നത്

pirakilaayaan
പിറകിലായാണ്
പിറകിലായാണ്

pankuvechayidam
പങ്കുവെച്ചയിടം
പങ്കുവെച്ചയിടം

kshethravarumaanavum
ക്ഷേത്രവരുമാനവും
ക്ഷേത്രവരുമാനവും

shambari

വിഭാഗീയതയൊന്നും

uthamamaayum
ഉത്തമമായും
ഉത്തമമായും

vaalakathulla
വാളകത്തുള്ള
വാളകത്തുള്ള

kozhinjuveezhukayum
കൊഴിഞ്ഞുവീഴുകയും
കൊഴിഞ്ഞുവീഴുകയും

maranappedunbol
മരണപ്പെടുന്പോൾ
മരണപ്പെടുന്പോൾ

purakilirunnaanu
പുറകിലിരുന്നാണ്
പുറകിലിരുന്നാണ്

keerunnoru
കീറുന്നൊരു
കീറുന്നൊരു

nasruddin
നസ്രുദ്ദീൻ
നസ്രുദ്ദിൻ

praanabhayatthode
പ്രാണഭയത്തോടെ
പ്രാണഭയത്തോടെ

sidhaarthanteyum
സിദ്ധാർത്ഥന്റെയും
സിദ്ധാർത്ഥന്റെയും

pariseelanashibiram
പരിശീലനശിബിരം
പരിശീലനശിബിരം

mattoralkku
മറ്റൊരാൾക്കു
മറ്റൊരാൾക്കു

prapanchasathyathinu
പ്രപഞ്ചസത്യത്തിനു
പ്രപഞ്ചസത്തത്തിന്

nolajiyumellaam
നോളജിയുമെല്ലാം
നോളജിയുമെല്ലാം

hyphayilulla
ഹൈഫയിലുള്ള
ഹൈഫയിലുള്ള

vikasippikkanaavum
വികസിപ്പിക്കാനാവും
വികസിപ്പിക്കാനാവും

ramakkalmettil
രാമക്കൽമേട്ടിൽ
രാമക്കം്ലേറ്റിൽ

rentalukalilekku
റെന്റലുകളിലേക്ക്
റെന്റലുകളിലേക്ക്

hackermarude
ഹാക്കർമാറുടെ
ഹാക്കർമാരുടെ

naalaramanivareykkum
നാലരമണിവരേയ്ക്കും
നാലരമണിവരെയ്ക്കും

bhaagyarude
ഭാഗ്യരുടെ
ഭാഗ്യരുടെ

uparifailamaaya
ഉപരിഫൈലമായ
ഉപരിഫൈലമായ

bruckup
ബ്രക്കപ്പ്
ബ്രക്കപ്പ്

k

രൂമിലിരുന്നാൽ

santhoshmaadhavante
സന്തോഷ്മാധവന്റെ
സന്തോഷ്മാധവന്റെ

doshamullavaraayi
ദോഷമുള്ളവരായി
ദോഷമുള്ളവരായി

chinthaapadhathikalkku
ചിന്താപദ്ധതികൾക്ക്
ചിന്താപദ്ധതികൾക്ക്

mediator
മീഡിയേറ്റർ
മീഡിയേറ്റർ

thirayunnavarkkulla
തിരയുന്നവർക്കുള്ള
തിരയുന്നവർക്കുള്ള

kandittillennu
കണ്ടിട്ടില്ലെന്നു
കണ്ടിട്ടില്ലെന്ന്

swadeshaabhimaaniye
സ്വദേശാഭിമാനിയെ
സ്വദേശാഭിമാനി

thadichirikkunnathu
തടിച്ചിരിക്കുന്നത്
തടിച്ചിരിക്കുന്നത്

pidichuparanju
പിടിച്ചുപറഞ്ഞു
പിടിച്ചുപറഞ്ഞ്

mandum
മണ്ടും
മണ്ടും

ethiraayippoyi
എതിരായിപ്പോയി
എതിരായിപ്പോയി

avabodhamillatheyum
അവബോധമില്ലാതെയും
അവബോധമില്ലാതെയും

empimaaradakkamullavare
എംപിമാരടക്കമുള്ളവരെ
എംപിമാരടക്കമുള്ളവരെ

rathikkaayi
രതിക്കായി
രതിക്കായി

onnucheraanulla
ഒന്നുചേരാനുള്ള
ഒന്നുചേരാനുള്ള

bhashaabhimaanikalil
ഭാഷാഭിമാനികളിൽ
ഭാഷാഭിമാനികളിൽ

thadanjuvachathukondu
തടഞ്ഞുവച്ചതുകൊണ്ട്
തടഞ്ഞുവച്ചതുകൊണ്ട്

thirikkukayundaayi
തിരിക്കുകയുണ്ടായി
തിരിക്കുകയുണ്ടായി

ponnamkandi
പൊന്നംകണ്ടി
പൊന്നംകണ്ടി

rimb
റിംബ്
റിമ്ബ്

pottasyatthinte
പൊട്ടസ്യത്തിന്റെ
പൊ

ഭാഗവതോത്തമാണ്

abhaavathiloode
അഭാവത്തിലൂടെ
അഭാവത്തിലൂടെ

pavaliyanileykkalla
പവലിയനിലേയ്ക്കല്ല
പവലിയനിലേയ്ക്കല്ല

vastradi
വസ്ത്രാദി
വസ്ത്രാദി

vegamkondum
വേഗംകൊണ്ടും
വേഗംകൊണ്ടും

janathinitharam
ജനത്തിനിത്തരം
ജനത്തിനിത്തരം

thejaswinik
തേജസ്വിനിക്ക്
തേജസ്വിനിക്

maattanamennokke
മാറ്റണമെന്നൊക്കെ
മാറ്റണമെന്നൊക്കെ

vottubaankine
വോട്ടുബാങ്കിനെ
വോട്ടുബാങ്കിനെ

madhyakaalakeralamaanu
മധ്യകാലകേരളമാണ്
മധ്ധാകകലേരകളാണ്

padatthileyum
പടത്തിലെയും
പദത്തിലെയും

vattukesine
വട്ടുകേസിനെ
വട്ടുകേസിനെ

paathavakkilekku
പാതവക്കിലേക്ക്
പാതവക്കിലേക്ക്

veethameduthum
വീതമെടുത്തും
വീതമെടുത്തും

fooclift
ഫോക്ലിഫ്റ്റ്
ഫോക്ലിഫ്റ്റ്

paattakkukayum
പാട്ടാക്കുകയും
പാറ്റക്കുകയും

himalayangalude
ഹിമാലയങ്ങളുടെ
ഹിമലയങ്ങളുടെ

oppuvechathumilla
ഒപ്പുവെച്ചതുമില്ല
ഒപ്പുവെച്ചതുമില്ല

prathiguruthvaakarshana
പ്രതിഗുരുത്വാകർഷണ
പ്രതിഗുരുത്വാകർഷൻ

tiwary
തീവാരി
തിവാരി

adikkanamenno
അടിക്കണമെന്നോ
അടിക്കണമെന്നോ

pachapuli
പച്ചപുളി
പച്ചപുളി

sunigay
സുനിഗയ്
സുനിഗയ്

chikithsaykkukoodi
ചികിത്സയ്ക്കുകൂടി
ചികിത്സയ്ക്കുകൂടി


IndexError: index 0 is out of bounds for axis 0 with size 0

In [37]:
def word_finder_eng(eng):
    eng_eos=np.where(eng==1)[0][0]
    eng_word=num_word(eng[0:eng_eos],eng_index2word)
    return eng_word
def word_finder_mal(mal):
    mal_eos=np.where(mal==1)[0][0]
    mal_word=num_word(mal[0:mal_eos],mal_index2word)
    return mal_word
def num_word(number,converter):
    number=np.array(number)
    word=''.join(converter[num]for num in number)
    return word

In [21]:
train_model(epochs=15,hidden_size=512,num_layers=2,encoder_drop_out=0.5,decoder_drop_out=0.2,embedding_size=32,
    bidirection=True,model='LSTM',lr=0.001,optimizer_='adam',teacher_force_ratio=0.5)

  0%|          | 0/800 [00:00<?, ?it/s]

epochs0 weights saved
Epochs 0 completed, train loss and accuracy =(0.7497897, 0.05779296875),and val loss and accuracy =(0.56165475, 0.25537109375) 


  0%|          | 0/800 [00:00<?, ?it/s]

epochs1 weights saved
Epochs 1 completed, train loss and accuracy =(0.22139466, 0.33578125),and val loss and accuracy =(0.5566614, 0.345947265625) 


  0%|          | 0/800 [00:00<?, ?it/s]

...

Epochs 12 completed, train loss and accuracy =(0.027791128, 0.85291015625),and val loss and accuracy =(0.64333844, 0.4296875) 


  0%|          | 0/800 [00:00<?, ?it/s]

Epochs 13 completed, train loss and accuracy =(0.025310943, 0.866484375),and val loss and accuracy =(0.7008637, 0.423583984375) 


  0%|          | 0/800 [00:00<?, ?it/s]

Epochs 14 completed, train loss and accuracy =(0.02232347, 0.87859375),and val loss and accuracy =(0.7243773, 0.423828125) 


In [19]:
def train_model(epochs=30,hidden_size=512,num_layers=3,encoder_drop_out=0.2,decoder_drop_out=0.2,embedding_size=256,
                bidirection=False,model='LSTM',lr=1e-3,optimizer_='adam',teacher_force_ratio=0.5):
    epochs=epochs
    hidden_size=hidden_size
    num_layers=num_layers
    encoder_drop_out=encoder_drop_out
    decoder_drop_out=decoder_drop_out
    embedding_size=embedding_size
    bidirection=bidirection
    model=model
    lr=lr
    optimizer_=optimizer_
    teacher_force_ratio=teacher_force_ratio

    encoder=Encoder(hidden_size=hidden_size,num_layers=num_layers,drop_out=encoder_drop_out,embedding_size=embedding_size,
                    bidirection=bidirection,model=model)
    decoder=Decoder(hidden_size=hidden_size,num_layers=num_layers,drop_out=decoder_drop_out,embedding_size=embedding_size,
                    bidirection=bidirection,model=model)
    model=seq2seq(encoder,decoder)
    model=model.to('cuda')
    lossfun=nn.CrossEntropyLoss()
    if optimizer_=='adam':
        optimizer=torch.optim.Adam(model.parameters(),lr=lr)
    elif optimizer_=='sgd':
        optimizer=torch.optim.SGD(model.parameters(),lr=lr)
    else:
        raise ValueError('Optimizer not found')
    train_loss,train_acc,val_loss,val_acc=[],[],[],[]
    curr_best_acc=0
    for i in range(epochs):
        train_loss_curr,train_acc_curr,val_loss_curr,val_acc_curr=[],[],[],[]
        model.train()
        for eng,mal in tqdm(train_loader):
            eng=eng.to('cuda')
            mal=mal.to('cuda')
            pred=model(eng,mal,teacher_force_ratio=teacher_force_ratio)
            #pred=nn.Softmax(dim=-1)(pred)
            pred_loss=pred.reshape(-1,pred.shape[2])
            mal_loss=mal.reshape(-1,).long()
            loss=lossfun(pred_loss,mal_loss)

            train_loss_curr.append(loss.cpu().detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            mal_ground=word_finder_mal(mal.cpu().detach().numpy())
            pred_acc=np.argmax(pred.cpu().detach().numpy(),axis=-1).astype(np.int64)
            mal_predicted=word_finder_mal(pred_acc)
            acc=np.sum(mal_ground==mal_predicted)/len(mal_ground)
            train_acc_curr.append(acc)
        model.eval()
        for eng,mal in test_loader:
            eng=eng.to('cuda')
            mal=mal.to('cuda')
            with torch.no_grad():
                pred=model(eng,mal,teacher_force_ratio=0)
            #pred=nn.Softmax(dim=-1)(pred)
            pred_loss=pred.reshape(-1,pred.shape[2])
            mal_loss=mal.reshape(-1,).long()

            loss=lossfun(pred_loss,mal_loss)
            val_loss_curr.append(loss.cpu().detach().numpy())


            mal_ground=word_finder_mal(mal.cpu().detach().numpy())
            pred_acc=np.argmax(pred.cpu().detach().numpy(),axis=-1)

            mal_predicted=word_finder_mal(pred_acc)
            acc=np.sum(mal_ground==mal_predicted)/len(mal_ground)
            val_acc_curr.append(acc)
        train_loss.append(np.average(train_loss_curr))
        val_loss.append(np.average(val_loss_curr))
        train_acc.append(np.average(train_acc_curr))
        val_acc.append(np.average(val_acc_curr))
        if val_acc[i]>curr_best_acc:
            torch.save(model.state_dict(),'weight_without_attention.pth')
            curr_best_acc=val_acc[i]
            print(f'epochs{i} weights saved')
        print(f'Epochs {i} completed, train loss and accuracy ={train_loss[i],train_acc[i]}' 
          f',and val loss and accuracy ={val_loss[i],val_acc[i]} ')
        '''
        wandb.log({"Train_Accuracy":np.round(train_acc[i]*100,2),"Train_Loss":train_loss[i],
                   "Val_Accuracy":np.round(val_acc[i]*100,2),"Val_Loss":val_loss[i],"Epoch":i+1})
        '''
    